In [14]:
import pandas as pd

df = pd.read_excel('/content/drive/MyDrive/2023_2024.xlsx')

df

,platform,track_name,artist_name,album_name,spotify_track_uri,reason_start,reason_end,shuffle,skipped,offline,minutes_played,date,month,year,hour,genres
0,ios,Wildwood Flower,The Carter Family,Can The Circle Be Unbroken: Country Music's Fi...,spotify:track:6LXh42hEpcVLKtUgVkiKZi,clickrow,unexpected-exit-while-paused,True,False,False,0.152167,2023-01-01,1,2023,15,"['bluegrass', 'traditional country', 'christia..."
1,ios,Easy On Me,Adele,Easy On Me,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,playbtn,remote,False,False,False,0.412800,2023-01-01,1,2023,17,['soft pop']
2,osx,Easy On Me,Adele,Easy On Me,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,remote,trackdone,True,False,False,3.348617,2023-01-01,1,2023,17,['soft pop']
3,osx,Empire State of Mind (Part II) Broken Down,Alicia Keys,The Element Of Freedom,spotify:track:5sra5UY6sD658OabHL3QtI,trackdone,trackdone,True,False,False,3.608000,2023-01-01,1,2023,17,"['r&b', 'neo soul']"
4,osx,champagne problems,Taylor Swift,evermore,spotify:track:0sY6ZUTh4yoctD8VIXz339,trackdone,trackdone,True,False,False,4.066667,2023-01-01,1,2023,17,['pop']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18085,ios,Teenagers,My Chemical Romance,The Black Parade,spotify:track:7j31rVgGX9Q2blT92VBEA0,trackdone,trackdone,True,False,False,2.698667,2024-12-31,12,2024,20,"['emo', 'pop punk']"
18086,ios,City,Kristina K,City,spotify:track:5ORwNYr0lREKs4vwQcpxh2,trackdone,fwdbtn,True,True,False,0.123167,2024-12-31,12,2024,20,"['indie folk', 'singer-songwriter']"
18087,ios,Noche Acosador,John 5,God Told Me to,spotify:track:4c3w4FIKZ7QvedYFPZalCF,fwdbtn,fwdbtn,True,True,False,1.126667,2024-12-31,12,2024,20,"['pop', 'soul']"
18088,ios,Girl on Fire,Alicia Keys,Girl on Fire (Remixes) - EP,spotify:track:4esOae7i4rqTbAu9o5Pxco,fwdbtn,trackdone,True,False,False,3.748667,2024-12-31,12,2024,20,"['r&b', 'neo soul']"


In [57]:
# Counting how many unique genre values
df['genres'].nunique()
#Too many!

1310

In [55]:
# Trying a K-mean clustering model

import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, calinski_harabasz_score
import matplotlib.pyplot as plt
import seaborn as sns
import ast
import json
from collections import defaultdict

# Defining genre similarities to help with preprocessing - after many attempts without genre_similarities that gave inconclusive results
GENRE_SIMILARITIES = {
    'rock_classic': ['classic rock', 'hard rock', 'album rock'],
    'rock_progressive': ['progressive rock', 'art rock', 'psychedelic rock'],
    'indie_rock': ['alternative rock', 'modern rock', 'indie rock', 'post-rock', 'garage rock', 'art rock'],
    'indie_pop': ['indie pop', 'chamber pop', 'dream pop', 'baroque pop'],
    'indie_folk': ['indie folk', 'singer-songwriter', 'indie'],
    'french_pop': ['french pop', 'variété française', 'chanson', 'french indie'],
    'soft_pop': ['soft pop', 'adult contemporary', 'easy listening'],
    'art_pop': ['art pop', 'experimental pop', 'avant-garde pop'],
    'rock': ['rock', 'indie rock', 'modern rock', 'classic rock', 'alternative rock', 'progressive rock', 'psychedelic rock', 'hard rock'],
    'electronic': ['electronic', 'edm', 'house', 'techno', 'ambient', 'idm', 'electronica', 'synthwave', 'drum and bass'],
    'rnb_soul': ['r&b', 'neo soul', 'soul', 'rnb', 'rhythm and blues', 'motown', 'funk'],
    'hip_hop': ['hip hop', 'rap', 'trap', 'drill', 'grime', 'conscious rap'],
    'jazz': ['jazz', 'bebop', 'swing', 'fusion', 'big band', 'cool jazz', 'modal jazz'],
    'classical': ['classical', 'orchestra', 'chamber', 'symphony', 'baroque', 'opera', 'contemporary classical'],
    'country': ['country', 'bluegrass', 'traditional country', 'christian country', 'americana', 'folk country'],
    'folk': ['folk', 'indie folk', 'traditional folk', 'contemporary folk', 'folk rock', 'singer songwriter'],
    'metal': ['metal', 'heavy metal', 'black metal', 'death metal', 'doom metal', 'progressive metal'],
    'punk': ['punk', 'punk rock', 'hardcore', 'post punk', 'pop punk', 'garage punk'],
    'world': ['world', 'latin', 'african', 'reggae', 'ska', 'celtic', 'asian', 'middle eastern'],
    'alternative': ['alternative', 'indie', 'experimental', 'art rock', 'post rock', 'noise'],
    'dance': ['dance', 'disco', 'house', 'techno', 'trance', 'electronica'],
    'blues': ['blues', 'rhythm and blues', 'chicago blues', 'delta blues', 'electric blues'],
    'guitar': ['acoustic guitar', 'fingerstyle', 'classical guitar', 'spanish guitar', 'flamenco', 'guitar virtuoso', 'instrumental guitar', 'guitar jazz'],
    'pop': ['pop'],
    'pop_rock': ['pop rock'],
    'latin_regional': ['regional mexican', 'corridos', 'mariachi', 'cumbia', 'nova mpb', 'sertanejo', 'tango', 'bolero'],
    'downtempo': ['trip hop', 'downtempo', 'lounge', 'slowcore', 'lullaby'],
    'world_traditional': ['sea shanties', 'choral', 'pan flute', 'andean', 'kora', 'malian', 'tuareg', 'bhangra']
}

# Defining genre weights
GENRE_WEIGHTS = {
    # Major genres get higher weights
    'rock': 1.5,
    'pop': 1.5,
    'hip_hop': 1.5,
    'electronic': 1.5,
    'jazz': 1.5,
    'classical': 1.5,

    # Secondary genres get medium weights
    'indie_rock': 1.2,
    'indie_pop': 1.2,
    'rnb_soul': 1.2,
    'metal': 1.2,
    'punk': 1.2,
    'folk': 1.2,
    'country': 1.2,
    'blues': 1.2,

    # Regional and niche genres get standard weights
    'world': 1.0,
    'latin_regional': 1.0,
    'downtempo': 1.0,
    'soft_pop': 1.0,
    'trip_hop': 1.0,
    'lounge': 1.0,
    'regional_mexican': 1.0,
    'tango': 1.0,
    'cumbia': 1.0,
    'nova_mpb': 1.0,
    'slowcore': 1.0,
    'choral': 1.0,
    'sea_shanties': 1.0,

    # Default weight for any unspecified genre
    'default': 1.0
}

# Creating reverse mapping for faster lookup
GENRE_MAPPING = {}
for main_genre, similar_genres in GENRE_SIMILARITIES.items():
    for genre in similar_genres:
        GENRE_MAPPING[genre] = main_genre

def standardize_genre(genre):
    # Standardizing genres
    genre = str(genre).lower().strip()
    genre = genre.replace('&', 'and')
    genre = genre.replace('-', ' ')
    genre = genre.replace('/', ' ')

    # Check for exact matches first
    if genre in GENRE_MAPPING:
        return GENRE_MAPPING[genre]

    # Check for partial matches, with longer matches taking the priority
    matches = []
    for similar_genre, main_genre in GENRE_MAPPING.items():
        if similar_genre in genre or genre in similar_genre:
            matches.append((similar_genre, main_genre, len(similar_genre)))

    if matches:
        # Sorting by length of match (longer matches first)
        matches.sort(key=lambda x: x[2], reverse=True)
        return matches[0][1]

    return genre

def preprocess_genres(genres_list):
    # Cleaning and standardize genre strings
    processed = []
    for genres in genres_list:
        try:
            # Handling null values
            if pd.isna(genres):
                processed.append([])
                continue

            # Converting string representation of list to actual list
            if isinstance(genres, str):
                try:
                    # Try to parse as JSON first
                    genres = json.loads(genres.replace("'", '"'))
                except json.JSONDecodeError:
                    # If that fails, try ast.literal_eval
                    try:
                        genres = ast.literal_eval(genres)
                    except (ValueError, SyntaxError):
                        # If still fails, treat as single genre
                        genres = [genres]

            # Ensure genres is a list
            if not isinstance(genres, list):
                genres = [genres]

            # Clean and standardize each genre
            cleaned_genres = []
            for genre in genres:
                if pd.isna(genre):
                    continue

                standardized = standardize_genre(genre)
                if standardized:
                    cleaned_genres.append(standardized)

            processed.append(cleaned_genres if cleaned_genres else ['unknown'])

        except Exception as e:
            print(f"Error processing genre entry: {genres}, Error: {e}")
            processed.append(['unknown'])

    return processed

def apply_genre_weights(X, genre_labels):
    """Apply weights to genres to give more importance to major genres."""
    weights = np.ones(len(genre_labels))

    for i, genre in enumerate(genre_labels):
        # Convert genre to format matching GENRE_WEIGHTS keys
        genre_key = genre.lower().replace(' ', '_').replace('-', '_')

        # Find the most similar key in GENRE_WEIGHTS
        matching_weight = GENRE_WEIGHTS.get(genre_key)
        if matching_weight is None:
            # Try to find a partial match
            for weight_key, weight_value in GENRE_WEIGHTS.items():
                if weight_key in genre_key or genre_key in weight_key:
                    matching_weight = weight_value
                    break

        weights[i] = matching_weight if matching_weight is not None else GENRE_WEIGHTS['default']

    # Apply weights to the feature matrix
    return X * weights

def analyze_cluster_genres(X, cluster_indices, genre_labels, top_n=5):
    """Analyze genres in a cluster and return top genres with their frequencies."""
    if len(cluster_indices) == 0:
        return []

    cluster_genre_counts = np.sum(X[cluster_indices], axis=0)

    # Get indices of top genres sorted by frequency
    top_indices = np.argsort(-cluster_genre_counts)[:top_n]

    # Get the actual genre names and their frequencies
    top_genres = [(genre_labels[i], cluster_genre_counts[i]) for i in top_indices if cluster_genre_counts[i] > 0]
    return top_genres

# Getting visualizations to download

def plot_cluster_sizes(df, save_path='cluster_sizes.png'):
    """Plot the distribution of cluster sizes."""
    plt.figure(figsize=(12, 6))
    cluster_sizes = df['cluster'].value_counts().sort_index()
    sns.barplot(x=cluster_sizes.index, y=cluster_sizes.values)
    plt.title('Distribution of cluster sizes')
    plt.xlabel('Cluster')
    plt.ylabel('Number of Tracks')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def plot_genre_distribution(X, genre_labels, save_path='genre_distribution.png'):
    #Plot the distribution of genres across the dataset
    plt.figure(figsize=(15, 8))
    genre_counts = X.sum(axis=0)
    sorted_indices = np.argsort(-genre_counts)[:20]  # Only show top 20 genres
    top_genres = genre_labels[sorted_indices]
    top_counts = genre_counts[sorted_indices]

    plt.bar(range(len(top_genres)), top_counts)
    plt.title('Distribution of top 20 genres in dataset')
    plt.xlabel('Genre')
    plt.ylabel('Frequency')
    plt.xticks(range(len(top_genres)), top_genres, rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(save_path)
    plt.close()

def evaluate_clusters(X, labels):
    #Evaluate clustering quality using multiple metrics
    if len(np.unique(labels)) <= 1:
        return None, None

    silhouette = silhouette_score(X, labels)
    calinski = calinski_harabasz_score(X, labels)

    return silhouette, calinski

def cluster_genres(df, n_clusters=25):

    #Main function to cluster genres and resolve primary genre for each track

    # Make a copy of the original genres
    df['original_genres'] = df['genres'].copy()

    # Step 1: Preprocess genres
    print("Preprocessing genres...")
    df['cleaned_genres'] = preprocess_genres(df['genres'])

    print("\nSample of preprocessing results:")
    print(df[['original_genres', 'cleaned_genres']].head())

    # Step 2: Multi-hot encoding
    print("\nEncoding genres...")
    mlb = MultiLabelBinarizer(sparse_output=False)
    genres_for_encoding = df['cleaned_genres'].tolist()
    X = mlb.fit_transform(genres_for_encoding)
    genre_labels = np.array(mlb.classes_)

    print(f"Number of unique genres after encoding: {len(genre_labels)}")
    print("Genre labels:", list(genre_labels))

    # Plot genre distribution
    plot_genre_distribution(X, genre_labels)

    # Apply weights to the genre features
    X_weighted = apply_genre_weights(X, genre_labels)

    # Step 3: Apply K-means clustering with weighted features
    print(f"\nApplying K-means clustering with {n_clusters} clusters...")
    kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
    kmeans.fit(X_weighted)
    df['cluster'] = kmeans.labels_

    # Evaluate clustering quality
    silhouette, calinski = evaluate_clusters(X_weighted, kmeans.labels_)
    print("\nClustering Quality Metrics:")
    print(f"Silhouette Score: {silhouette:.3f} (ranges from -1 to 1, higher is better)")
    print(f"Calinski-Harabasz Score: {calinski:.3f} (higher is better)")

    # Plot cluster sizes
    plot_cluster_sizes(df)

    # Step 4: Analyze clusters
    print("\nAnalyzing clusters...")
    cluster_genre_mapping = {}

    for cluster in range(n_clusters):
        cluster_indices = np.where(kmeans.labels_ == cluster)[0]
        cluster_size = len(cluster_indices)
        top_genres = analyze_cluster_genres(X, cluster_indices, genre_labels, top_n=5)
        cluster_genre_mapping[cluster] = top_genres

        # Try to identify the cluster type based on top genres
        cluster_type = "Mixed"
        if top_genres:
            main_genre = top_genres[0][0]
            # Check for matches in genre similarities
            for genre_type, similar_genres in GENRE_SIMILARITIES.items():
                if main_genre in similar_genres or any(sg in main_genre or main_genre in sg for sg in similar_genres):
                    cluster_type = genre_type.replace('_', ' ').title()
                    break

        print(f"\nCluster {cluster} ({cluster_type}):")
        print(f"Number of tracks: {cluster_size} ({(cluster_size/len(df)*100):.1f}% of total)")
        print("Top genres (with frequencies):")
        for genre, freq in top_genres:
            print(f"  - {genre}: {int(freq)} ({(freq/cluster_size*100):.1f}% of cluster)")

        # Print sample of original genres in this cluster
        sample_indices = np.random.choice(cluster_indices, size=min(5, len(cluster_indices)), replace=False)
        print("Sample of original genres in this cluster:")
        for idx in sample_indices:
            print(f"  {df['original_genres'].iloc[idx]}")

    return df

def create_custom_clusters(df, genre_rules):
    """
    Create new clusters based on genre string matching rules.

    Args:
        df: DataFrame with 'cleaned_genres' and 'cluster' columns
        genre_rules: Dictionary where key is new cluster name and value is list of genres to match
    """
    # Make a copy of the original cluster assignment
    df['original_cluster'] = df['cluster']

    # Applying each rule
    for new_cluster, genres_to_match in genre_rules.items():
        # Create mask for genres matching any of the rules
        mask = df['cleaned_genres'].apply(
            lambda x: any(any(genre in str(g).lower() for g in x) for genre in genres_to_match)
        )

        # Assign new cluster where mask is True
        df.loc[mask, 'cluster'] = new_cluster

        # Print statistics about the new cluster
        n_tracks = mask.sum()
        if n_tracks > 0:
            print(f"\nCreated new cluster '{new_cluster}':")
            print(f"Number of tracks: {n_tracks}")
            print("Sample tracks and their genres:")
            sample = df[mask].sample(min(3, n_tracks))
            for _, row in sample.iterrows():
                print(f"  Original genres: {row['original_genres']}")
                print(f"  Original cluster: {row['original_cluster']}")
                print(f"  New cluster: {row['cluster']}\n")

    return df


if __name__ == "__main__":
    try:
        # Perform genre clustering
        df = cluster_genres(df, n_clusters=25)
        print("\nSample of results:")
        print(df[['original_genres', 'cleaned_genres', 'cluster']].head())

        # Adding a few rules because the clustering is not conclusive enough

        # Creating custom clusters
        genre_rules = {
            'other world': ['world', 'reggae', 'afro', 'celtic',
            'african', 'asian', 'indian', 'bollywood', 'chinese', 'japanese', 'korean',
            'middle eastern', 'arabic', 'persian', 'turkish', 'balkan',
            'caribbean', 'jamaican', 'calypso', 'fado', 'klezmer',
            'polynesian', 'hawaiian', 'andalusian', 'romani', 'gamelan', 'aboriginal',
            'traditional', 'ethnic', 'folk music', 'indigenous'],
            'latin': ['latin', 'salsa', 'bossa', 'bossa nova', 'samba', 'latin jazz', 'latin pop', 'reggaeton', 'bachata', 'brazilian', 'cuban'],
            'jazz': ['jazz', 'bebop', 'swing', 'big band', 'cool jazz', 'modal jazz', 'jazz fusion',
             'jazz funk', 'jazz rock', 'smooth jazz', 'free jazz', 'contemporary jazz'],
            'christmas': ['christmas'],
            'metal': ['metal'],
            'country': ['country', 'bluegrass', 'traditional country', 'christian country', 'americana', 'folk country']
        }

        df = create_custom_clusters(df, genre_rules)

        print("\nVisualization files generated:")
        print("- cluster_sizes.png: Shows the distribution of tracks across clusters")
        print("- genre_distribution.png: Shows the overall distribution of genres in the dataset")

    except Exception as e:
        print(f"Error processing genres: {e}")

Preprocessing genres...

Sample of preprocessing results:
                                     original_genres  \
0  ['bluegrass', 'traditional country', 'christia...   
1                                       ['soft pop']   
2                                       ['soft pop']   
3                                ['r&b', 'neo soul']   
4                                            ['pop']   

                cleaned_genres  
0  [country, country, country]  
1                   [soft_pop]  
2                   [soft_pop]  
3            [randb, rnb_soul]  
4                        [pop]  

Encoding genres...
Number of unique genres after encoding: 232
Genre labels: ["['brazilian',\xa0'guitar']", "['children's music', 'german']", 'acoustic\xa0guitar', 'adult standards', 'afro adura', 'afrobeat', 'afrobeats', 'agronejo', 'algerian chaabi', 'alt randb', 'alternative', 'alté', 'amapiano', 'anime', 'arabic', 'argentine', 'arrocha', 'art_pop', 'asmr', 'avant garde', 'azonto', 'bachata', 'balkan

<ipython-input-55-8a63e1121480>:331: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'other world' has dtype incompatible with int32, please explicitly cast to a compatible dtype first.
  df.loc[mask, 'cluster'] = new_cluster



Created new cluster 'other world':
Number of tracks: 1194
Sample tracks and their genres:
  Original genres: ['latin pop', 'ranchera'] 
  Original cluster: 17
  New cluster: other world

  Original genres: ['latin pop', 'dance'] 
  Original cluster: 17
  New cluster: other world

  Original genres: ['celtic']
  Original cluster: 17
  New cluster: other world


Created new cluster 'latin':
Number of tracks: 577
Sample tracks and their genres:
  Original genres: ['corridos tumbados', 'sad sierreño', 'corrido', 'sierreño', 'música mexicana', 'corridos bélicos']
  Original cluster: 22
  New cluster: latin

  Original genres: ['bolero']
  Original cluster: 22
  New cluster: latin

  Original genres: ['classical', 'mexican'] 
  Original cluster: 5
  New cluster: latin


Created new cluster 'jazz':
Number of tracks: 755
Sample tracks and their genres:
  Original genres:  ['soul', 'r&b', 'jazz']
  Original cluster: 9
  New cluster: jazz

  Original genres: ['jazz blues', 'soul', 'soul blues',

In [46]:
df.head(50)

# Looking at the new columns, some matching do not make sense

,platform,track_name,artist_name,album_name,spotify_track_uri,reason_start,reason_end,shuffle,skipped,offline,minutes_played,date,month,year,hour,genres,original_genres,cleaned_genres,cluster,original_cluster
0,ios,Wildwood Flower,The Carter Family,Can The Circle Be Unbroken: Country Music's Fi...,spotify:track:6LXh42hEpcVLKtUgVkiKZi,clickrow,unexpected-exit-while-paused,True,False,False,0.152167,2023-01-01,1,2023,15,"['bluegrass', 'traditional country', 'christia...","['bluegrass', 'traditional country', 'christia...","[country, country, country]",country,17
1,ios,Easy On Me,Adele,Easy On Me,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,playbtn,remote,False,False,False,0.412800,2023-01-01,1,2023,17,['soft pop'],['soft pop'],[soft_pop],17,17
2,osx,Easy On Me,Adele,Easy On Me,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,remote,trackdone,True,False,False,3.348617,2023-01-01,1,2023,17,['soft pop'],['soft pop'],[soft_pop],17,17
3,osx,Empire State of Mind (Part II) Broken Down,Alicia Keys,The Element Of Freedom,spotify:track:5sra5UY6sD658OabHL3QtI,trackdone,trackdone,True,False,False,3.608000,2023-01-01,1,2023,17,"['r&b', 'neo soul']","['r&b', 'neo soul']","[randb, rnb_soul]",9,9
4,osx,champagne problems,Taylor Swift,evermore,spotify:track:0sY6ZUTh4yoctD8VIXz339,trackdone,trackdone,True,False,False,4.066667,2023-01-01,1,2023,17,['pop'],['pop'],[pop],6,6
5,osx,Rainbow,Kacey Musgraves,Golden Hour,spotify:track:79qxwHypONUt3AFq0WPpT9,trackdone,endplay,True,True,False,0.830633,2023-01-01,1,2023,17,"['country dawn, contemporary country, classic ...","['country dawn, contemporary country, classic ...",[country],country,17
6,osx,"Elgar: Cello Concerto in E Minor, Op. 85: I. A...",Edward Elgar,Impressions,spotify:track:5pjDwtMPly0cNMbuGXhUOI,playbtn,trackdone,True,False,False,7.976217,2023-01-01,1,2023,17,"['classical', 'orchestra']","['classical', 'orchestra']","[classical, classical]",5,5
7,osx,Il barbiere di Siviglia: Overture (Sinfonia),Gioachino Rossini,Rossini: Complete Overtures,spotify:track:1chTrqszWQEOP4RsNuOZf7,trackdone,trackdone,True,False,False,7.083333,2023-01-01,1,2023,17,"['classical', 'opera']","['classical', 'opera']","[classical, classical]",5,5
8,osx,"Serenade for Strings in E Major, Op. 22, B. 52...",Antonín Dvořák,"Dvořák: Serenade for Strings, Czech Suite",spotify:track:3sAYxq1986j3ydqLv6jwUJ,trackdone,trackdone,True,False,False,7.208883,2023-01-01,1,2023,17,"['classical', 'chamber music', 'orchestra']","['classical', 'chamber music', 'orchestra']","[classical, classical, classical]",5,5
9,osx,"Violin Concerto in E Minor, Op. 64, MWV O 14: ...",Felix Mendelssohn,"Mendelssohn: Violin Concerto Symphony No.4 - ""...",spotify:track:4Ivo07fh9EyPtPinrxXlC4,trackdone,trackdone,True,False,False,12.129767,2023-01-01,1,2023,18,"['chamber music', 'classical', 'classical pian...","['chamber music', 'classical', 'classical pian...","[classical, classical, classical, world_tradit...",other world,5


In [56]:
# Dowloading the file to examine the mismatching

df.to_csv('clustered_genresnew.csv', index=False)

In [58]:
# Getting a list of all unique genres to find a new way to classify instead of clustering
print(df['genres'].unique())

["['bluegrass', 'traditional country', 'christian country']"
 "['soft pop']" "['r&b', 'neo soul']" ... "['neo soul', 'r&b'] "
 "['hip hop', 'uk rap'] " "['r&b', 'electronic'] "]


In [100]:
# Using 3.5 Sonnet AI, I configurated a new genre classification

import ast
import re

# Copy all the functions from improved_genre_rules.py
def clean_genres_improved(df):
    """
    Improved genre cleaning with comprehensive genre categories
    """
    # Primary genre categories with their subgenres
    genre_hierarchy = {
        'rock': {
            'genres': ['rock', 'classic rock', 'hard rock', 'indie rock', 'psychedelic rock',
                      'progressive rock', 'art rock', 'garage rock', 'glam rock', 'southern rock',
                      'surf rock', 'blues rock', 'folk rock', 'acid rock', 'british rock',
                      'heartland rock', 'piano rock', 'funk rock', 'modern rock', 'noise rock', 'space rock',
                      'stoner rock', 'roots rock', 'acoustic rock', 'album rock', 'rockabilly', 'rock and roll',
                      'symphonic rock', 'french rock', 'reggae rock', 'electro rock', 'k-rock', 'brazilian rock',
                      'australian rock', 'rock en español', 'blues rock'],
            'priority': 1
        },
        'pop': {
            'genres': ['pop','power pop', 'britpop'],
            'priority': 1
        },
        'asmr': {
            'genres': ['asmr', 'ASMR', 'meditation', 'relaxation', 'sleep', 'sleep music',
                      'nature sounds', 'white noise', 'ambient sleep', 'relaxing music'],
            'priority': 1
        },
        'dance': {
            'genres': ['dance', 'dance pop', 'dance-pop', 'alternative dance', 'latin dance', 'italo dance',
                      'electronic dance', 'uk garage', 'disco', 'post-disco', 'nu disco', 'disco house'],
            'priority': 1
        },
        'folk': {
            'genres': ['folk', 'indie folk', 'folk pop', 'traditional folk', 'psychedelic folk',
                      'folk punk', 'acoustic folk', 'ambient folk', 'american folk', 'british folk',
                      'french folk', 'world folk', 'caucasian folk', 'singer-songwriter', '60s folk',
                      'alternative folk', 'traditional music', 'sea shanties'],
            'priority': 1
        },
        'neoclassical': {
            'genres': ['neoclassical', 'contemporary classical', 'minimalism', 'minimalist', 'avant-garde',
                      'modern classical', 'contemporary piano', 'new age', 'piano'],
            'priority': 1
        },
        'hip hop': {
            'genres': ['hip hop', 'hiphop', 'rap', 'hip-hop', 'trap', 'conscious rap', 'jazz rap',
                      'alternative hip hop', 'experimental hip hop', 'gangster rap', 'g funk', 'pop rap',
                      'cloud rap', 'emo rap', 'uk hip hop', 'french rap', 'norwegian rap', 'german hip hop',
                      'malayalam hip hop', 'desi hip hop', 'boom bap', 'grime', 'uk drill', 'drill',
                      'rage rap', 'industrial hip hop', 'alternative rap', 'rap metal'],
            'priority': 1
        },
        'french pop': {
            'genres': ['french pop', 'variété française', 'chanson', 'french house', 'french jazz',
                      'pop urbaine', 'nouvelle chanson française', 'french indie'],
            'priority': 1
        },
        'soul / rnb': {
            'genres': ['soul', 'rnb', 'r&b', 'soul / rnb', 'neo soul', 'contemporary r&b',
                      'rhythm and blues', 'soul jazz', 'soul blues', 'northern soul', 'retro soul',
                      'soul pop', 'gospel r&b', 'indie soul', 'alternative r&b', 'indie r&b',
                      'motown', 'doo-wop', 'quiet storm', 'gospel', 'funk', 'afro soul'],
            'priority': 1
        },
        'pop rock': {
            'genres': ['pop rock', 'soft rock', 'power pop', 'electropop', 'pop alternative',
                      'british pop', 'belgian pop', 'dutch pop', 'german pop', 'georgian pop',
                      'hindi pop', 'dark pop', 'k-pop'],
            'priority': 1
        },
        'soft pop': {
            'genres': ['soft pop'],
            'priority': 1
        },
        'house': {
            'genres': ['house', 'deep house', 'french house', 'tropical house', 'funky house',
                      'disco house', 'organic house', 'tribal house', 'stutter house', 'lo-fi house',
                      'afro house', 'tech house'],
            'priority': 1
        },
        'indie pop / folk': {
            'genres': ['indie pop', 'indie pop / alt', 'alt', 'indie', 'alternative', 'bedroom pop',
                      'dream pop', 'chamber pop', 'baroque pop', 'art pop', 'indie folk',
                      'alternative pop', 'jangle pop', 'shoegaze', 'slowcore', 'dreampop', 'indietronica',
                      'chillwave', 'lo-fi', 'hyperpop'],
            'priority': 1
        },
        'guitar': {
            'genres': ['guitar', 'classical guitar', 'spanish guitar', 'acoustic guitar', 'fingerstyle',
                      'acoustic'],
            'priority': 1
        },
        'alt rock': {
            'genres': ['alt rock', 'alternative rock', 'indie rock', 'grunge', 'post-grunge',
                      'post-punk', 'new wave', 'madchester', 'emo', 'midwest emo', 'indie rock'],
            'priority': 1
        },
        'world music': {
            'genres': ['world music', 'world', 'african', 'celtic', 'indian', 'arabic', 'brazilian',
                      'traditional music', 'traditional mandinka', 'japanese traditional', 'greek folk',
                      'turkish', 'persian', 'tibetan', 'bollywood', 'desi', 'afrobeats', 'afrobeat',
                      'afropop', 'azonto', 'afroswing', 'alté', 'amapiano', 'gnawa', 'raï', 'qawwali',
                      'sufi', 'khaleeji', 'sholawat'],
            'priority': 1
        },
        'latin music': {
            'genres': ['latin music', 'latin', 'salsa', 'bachata', 'reggaeton', 'merengue', 'cumbia',
                      'tango', 'bossa nova', 'samba', 'latin pop', 'latin rock', 'latin jazz',
                      'latin alternative', 'latin indie', 'latin folk', 'latin urban', 'latin trap',
                      'latin r&b', 'bolero', 'cha cha cha', 'son cubano', 'trova', 'ranchera',
                      'mariachi', 'corridos', 'cumbia norteña', 'mambo', 'reggae en español',
                      'corridos tumbados', 'sierreño', 'música mexicana'],
            'priority': 1
        },
        'electronic': {
            'genres': ['electronic', 'edm', 'techno', 'electronica', 'ambient', 'synthpop',
                      'drum and bass', 'dubstep', 'trance', 'electro', 'downtempo', 'trip hop',
                      'idm', 'breakbeat', 'hardstyle', 'hardcore techno', 'gabber', 'industrial',
                      'dark ambient', 'minimal techno', 'future bass', 'electro swing', 'chiptune',
                      'darkwave', 'industrial metal', 'new rave', 'hi-nrg', 'jersey club'],
            'priority': 1
        },
        'classical': {
            'genres': ['classical', 'orchestra', 'chamber music', 'classical piano', 'opera', 'choral',
                      'baroque', 'romantic era', 'orchestral', 'chamber', 'requiem', 'early music',
                      'ukrainian classical', 'italian classical', 'cuban classical', 'spanish classical',
                      'greek classical', 'japanese classical', 'ballet', 'cinematic'],
            'priority': 1
        },
        'punk': {
            'genres': ['punk', 'pop punk', 'hardcore punk', 'post punk', 'punk rock' 'folk punk', 'skate punk',
                      'celtic punk', 'proto punk', 'garage punk', 'art punk', 'punk blues', 'punk rap',
                      'emo pop', 'hardcore'],
            'priority': 1
        },
        'soundtrack': {
            'genres': ['soundtrack', 'film score', 'musical theatre', 'film music', 'movie soundtrack',
                      'video game music', 'anime', 'cartoon soundtrack', 'silent era', 'cinematic',
                      'musicals', 'instrumental', 'video game', 'chiptune', 'gaming'],
            'priority': 1
        },
        'jazz': {
            'genres': ['jazz', 'bebop', 'swing', 'jazz fusion', 'smooth jazz', 'big band', 'cool jazz',
                      'modal jazz', 'jazz funk', 'jazz blues', 'vocal jazz', 'gypsy jazz', 'jazz manouche',
                      'nu jazz', 'acid jazz', 'jazz pop', 'french jazz', 'latin jazz', 'soul jazz',
                      'contemporary jazz', 'brazilian jazz', 'lounge'],
            'priority': 1
        },
        'metal': {
            'genres': ['metal', 'heavy metal', 'black metal', 'death metal', 'doom metal', 'power metal',
                      'progressive metal', 'thrash metal', 'gothic metal', 'folk metal', 'industrial metal',
                      'symphonic metal', 'melodic death metal', 'metalcore', 'nu metal', 'sludge metal',
                      'groove metal', 'pirate metal', 'medieval metal', 'alternative metal', 'glam metal'],
            'priority': 1
        },
        'country': {
            'genres': ['country', 'bluegrass', 'americana', 'traditional country', 'country rock',
                      'alt country', 'contemporary country', 'country pop', 'country folk',
                      'outlaw country', 'country rap', 'red dirt', 'gothic country', 'christian country',
                      'classic texas country'],
            'priority': 1
        },
        'christmas': {
            'genres': ['christmas', 'holiday music', 'christmas music', 'christmas pop', 'christmas jazz',
                      'christmas folk', 'christmas rock', 'christmas classical', 'ccm', 'christian'],
            'priority': 1
        }
    }

    def get_primary_genre(genres_list):
        """
        Determine the primary genre based on hierarchy and priority
        """
        if isinstance(genres_list, str):
            try:
                genres_list = ast.literal_eval(genres_list)
            except:
                genres_list = [genres_list]

        # Convert to lowercase and split any combined genres
        expanded_genres = []
        for genre in genres_list:
            expanded_genres.extend([g.strip().lower() for g in genre.split(',')])

        # Find matching primary genres and their priorities
        matches = []
        for primary, details in genre_hierarchy.items():
            for genre in details['genres']:
                for expanded_g in expanded_genres:
                    if genre.lower() == expanded_g:  # Changed to exact match
                        matches.append((primary, details['priority']))
                        break

        if not matches:
            # Try partial match if no exact match found
            for primary, details in genre_hierarchy.items():
                for genre in details['genres']:
                    for expanded_g in expanded_genres:
                        if genre.lower() in expanded_g or expanded_g in genre.lower():
                            matches.append((primary, details['priority']))
                            break

        if not matches:
            return 'other'

        # Return the match with highest priority (lowest number)
        return min(matches, key=lambda x: x[1])[0]

    # Apply the improved genre cleaning
    df['improved_cluster'] = df['cleaned_genres'].apply(get_primary_genre)

    # Additional post-processing rules
    def apply_post_rules(row):
        genres = row['cleaned_genres']
        current_cluster = row['improved_cluster']

        # Convert string representation to list if necessary
        if isinstance(genres, str):
            try:
                genres = ast.literal_eval(genres)
            except:
                genres = [genres]

        # Special case rules
        genres_str = str(genres).lower()

        if 'children' in genres_str:
            return 'children'
        if any(g in genres_str for g in ['comedy']):
            return 'comedy'

        return current_cluster

    df['improved_cluster'] = df.apply(apply_post_rules, axis=1)

    return df

def analyze_clusters(df):
    """
    Analyze the distribution of genres in each cluster
    """
    cluster_analysis = {}

    for cluster in df['improved_cluster'].unique():
        cluster_data = df[df['improved_cluster'] == cluster]
        all_genres = []

        for genres in cluster_data['cleaned_genres']:
            if isinstance(genres, str):
                try:
                    genres = ast.literal_eval(genres)
                except:
                    genres = [genres]
            all_genres.extend(genres)

        genre_counts = pd.Series(all_genres).value_counts()
        cluster_analysis[cluster] = {
            'top_genres': genre_counts.head(5).to_dict(),
            'total_tracks': len(cluster_data)
        }

    return cluster_analysis

# Main execution
if __name__ == "__main__":

    # Clean the genres
    df = clean_genres_improved(df)

    # Analyze clusters
    analysis = analyze_clusters(df)

    # Print analysis
    print("\nCluster Analysis:")
    for cluster, info in analysis.items():
        print(f"\nCluster: {cluster}")
        print(f"Total tracks: {info['total_tracks']}")
        print("Top genres:", info['top_genres'])

    # Save the results
    df.to_csv('improved_clusters.csv', index=False)
    print("\nResults saved to 'improved_clusters.csv'")



Cluster Analysis:

Cluster: country
Total tracks: 75
Top genres: {'country': 143, 'red dirt': 8, 'blues': 5, 'hip_hop': 3, 'christmas': 1}

Cluster: pop
Total tracks: 4173
Top genres: {'pop': 2854, 'french_pop': 1661, 'indie_pop': 1075, 'pop_rock': 317, 'dance': 208}

Cluster: soul / rnb
Total tracks: 263
Top genres: {'rnb_soul': 349, 'randb': 148, 'hip_hop': 26, 'quiet storm': 15, 'gospel': 7}

Cluster: classical
Total tracks: 775
Top genres: {'classical': 1220, 'jazz': 132, 'soundtrack': 92, 'world_traditional': 39, 'cinematic': 39}

Cluster: rock
Total tracks: 4149
Top genres: {'rock': 5941, 'alternative': 1231, 'folk': 376, 'pop': 363, 'metal': 258}

Cluster: soundtrack
Total tracks: 192
Top genres: {'soundtrack': 187, 'anime': 48, 'musical theatre': 17, 'film music': 16, 'musicals': 9}

Cluster: punk
Total tracks: 208
Top genres: {'punk': 298, 'indie_pop': 37, 'metal': 14, 'hip_hop': 6, 'indie_rock': 5}

Cluster: dance
Total tracks: 1288
Top genres: {'french_pop': 1472, 'dance': 

In [101]:
def recategorize_other(cluster, genres_str):
        """
        Recategorize tracks that ended up in 'other' based on their genres
        """
        if cluster != 'other':
            return cluster

        genres_str = str(genres_str).lower()

        # Handling the cases found in 'other'
        if 'french_pop' in genres_str:
            return 'french pop'
        if any(g in genres_str for g in ['hip_hop', 'hip hop', 'hiphop', 'rap']):
            return 'hip hop'
        if any(g in genres_str for g in ['randb', 'r&b', 'rnb']):
            return 'soul / rnb'
        if any(g in genres_str for g in ['soft_pop', 'soft pop']):
            return 'pop rock'
        if any(g in genres_str for g in ['doo wop', 'doo-wop']):
            return 'soul / rnb'

        return 'other'

df['improved_cluster'] = df.apply(lambda row: recategorize_other(row['improved_cluster'], row['cleaned_genres']), axis=1)

In [102]:
df

,platform,track_name,artist_name,album_name,spotify_track_uri,reason_start,reason_end,shuffle,skipped,offline,...,date,month,year,hour,genres,original_genres,cleaned_genres,cluster,original_cluster,improved_cluster
0,ios,Wildwood Flower,The Carter Family,Can The Circle Be Unbroken: Country Music's Fi...,spotify:track:6LXh42hEpcVLKtUgVkiKZi,clickrow,unexpected-exit-while-paused,True,False,False,...,2023-01-01,1,2023,15,"['bluegrass', 'traditional country', 'christia...","['bluegrass', 'traditional country', 'christia...","[country, country, country]",country,17,country
1,ios,Easy On Me,Adele,Easy On Me,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,playbtn,remote,False,False,False,...,2023-01-01,1,2023,17,['soft pop'],['soft pop'],[soft_pop],17,17,pop
2,osx,Easy On Me,Adele,Easy On Me,spotify:track:0gplL1WMoJ6iYaPgMCL0gX,remote,trackdone,True,False,False,...,2023-01-01,1,2023,17,['soft pop'],['soft pop'],[soft_pop],17,17,pop
3,osx,Empire State of Mind (Part II) Broken Down,Alicia Keys,The Element Of Freedom,spotify:track:5sra5UY6sD658OabHL3QtI,trackdone,trackdone,True,False,False,...,2023-01-01,1,2023,17,"['r&b', 'neo soul']","['r&b', 'neo soul']","[randb, rnb_soul]",9,9,soul / rnb
4,osx,champagne problems,Taylor Swift,evermore,spotify:track:0sY6ZUTh4yoctD8VIXz339,trackdone,trackdone,True,False,False,...,2023-01-01,1,2023,17,['pop'],['pop'],[pop],6,6,pop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18085,ios,Teenagers,My Chemical Romance,The Black Parade,spotify:track:7j31rVgGX9Q2blT92VBEA0,trackdone,trackdone,True,False,False,...,2024-12-31,12,2024,20,"['emo', 'pop punk']","['emo', 'pop punk']","[emo, punk]",14,14,alt rock
18086,ios,City,Kristina K,City,spotify:track:5ORwNYr0lREKs4vwQcpxh2,trackdone,fwdbtn,True,True,False,...,2024-12-31,12,2024,20,"['indie folk', 'singer-songwriter']","['indie folk', 'singer-songwriter']","[folk, folk]",4,4,folk
18087,ios,Noche Acosador,John 5,God Told Me to,spotify:track:4c3w4FIKZ7QvedYFPZalCF,fwdbtn,fwdbtn,True,True,False,...,2024-12-31,12,2024,20,"['pop', 'soul']","['pop', 'soul']","[pop, rnb_soul]",6,6,pop
18088,ios,Girl on Fire,Alicia Keys,Girl on Fire (Remixes) - EP,spotify:track:4esOae7i4rqTbAu9o5Pxco,fwdbtn,trackdone,True,False,False,...,2024-12-31,12,2024,20,"['r&b', 'neo soul']","['r&b', 'neo soul']","[randb, rnb_soul]",9,9,soul / rnb


In [103]:
# Making sure the 'other' cluster is drastically reduced
df['improved_cluster'].value_counts()

,count
improved_cluster,
pop,4173
rock,4149
folk,1517
dance,1288
indie pop / folk,987
asmr,777
classical,775
guitar,671
hip hop,608


In [104]:
# Removing unnecessary columns
df.drop(columns=['original_genres', 'cleaned_genres', 'cluster', 'original_cluster'], inplace=True)

In [105]:
# Exporting csv file for SQL
df.to_csv('improved_clusters.csv', index=False)